In [72]:
import torch

In [73]:
print(torch.cuda.is_available())

True


In [74]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from tqdm import tqdm
torch.manual_seed(21)

Hyperparameters

In [75]:
## DO NOT DELETE
# HYPERPARAMETERS
batch_size = 64
learning_rate = 1e-3
num_epochs = 5
C = 1.0

Preprocessing

In [76]:
## DO NOT DELETE
# PREPROCESSING
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [77]:
### DO NOT DELETE ###
def evaluate(model, data_loader, method=None):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in data_loader:
            if method == 'lr':
              outputs = model(data.view(-1,28*28))
            else:
              outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return 100 * correct / total

# LOGISTIC REGRESSION

In [78]:
import math

In [79]:
data, target = next(iter(train_loader))

data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [80]:
## DO NOT DELETE
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        # TODO implement LR model here
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)  
    def forward(self, x):
        # TODO implement forward pass here
        x = x.view(x.size(0), -1) #Flatten the input tensor
        z = self.linear(x) #linear transformation
        return z        

In [81]:
## DO NOT DELETE
lr_model = LogisticRegression(input_size= 28 * 28 , num_classes= 10)
loss_fn = nn.CrossEntropyLoss() #multiclass log softmax function
lr_optimizer = torch.optim.Adam(lr_model.parameters(), lr=learning_rate)

In [82]:
lr_losses = []
for epoch in tqdm(range(num_epochs)):
    lr_model.train()
    lr_loss_per_epoch = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # Train LR model
        # TODO Implement the lr model training loop here
        pred = lr_model(data) #X = data
        loss = loss_fn(pred, target)

        # loss.backward()
        lr_optimizer.zero_grad()
        loss.backward()
        lr_optimizer.step()
        lr_loss_per_epoch += loss.item()

    lr_losses.append(lr_loss_per_epoch / len(train_loader))
    print(f'Epoch {epoch+1}/{num_epochs}, NN Loss: {lr_losses[-1]}')

 20%|██        | 1/5 [00:08<00:33,  8.28s/it]

Epoch 1/5, NN Loss: 0.3909980665916192


 40%|████      | 2/5 [00:16<00:24,  8.14s/it]

Epoch 2/5, NN Loss: 0.2937971075007847


 60%|██████    | 3/5 [00:24<00:16,  8.26s/it]

Epoch 3/5, NN Loss: 0.28207019655339755


 80%|████████  | 4/5 [00:33<00:08,  8.45s/it]

Epoch 4/5, NN Loss: 0.2751435860673756


100%|██████████| 5/5 [00:42<00:00,  8.45s/it]

Epoch 5/5, NN Loss: 0.27088627276787247


In [83]:
# TODO Implement the LR evaluation code
# Hint : use the `evaluate` function above on the test set
evaluate(lr_model, test_loader)

92.38

# SVM

In [84]:
## DO NOT DELETE
class SVM(nn.Module):

    def __init__(self, input_size, num_classes):
        # TODO implement SVM model here
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes, bias = False)
        nn.LogSigmoid

    def forward(self, x):
        # TODO implement the forward pass here
        x = x.view(x.size(0), -1)
        return self.linear(x)

In [85]:
## DO NOT DELETE
# SVM MODEL AND OPTIMIZER
svm_model = SVM(input_size= 28 * 28, num_classes= 10)
svm_optimizer = optim.SGD(svm_model.parameters(), lr=learning_rate)

In [86]:
## DO NOT DELETE
# HINGE LOSS FOR SVM
def hinge_loss(outputs, labels):
    num_samples = outputs.size(0)
    correct_scores = outputs[torch.arange(num_samples), labels].unsqueeze(1)
    margins = torch.clamp(1 - (correct_scores - outputs), min=0)
    margins[torch.arange(num_samples), labels] = 0
    loss = torch.sum(margins) / num_samples
    return loss

In [87]:
svm_losses, ls_svm_losses = [], []

for epoch in tqdm(range(num_epochs)):
    svm_model.train()
    svm_loss_per_epoch = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # Train SVM
        # TODO Implement the SVM model training loop

        pred = svm_model(data)
        loss = hinge_loss(pred, target) #calculating the SVM hinge loss

        svm_optimizer.zero_grad()
        loss.backward()
        svm_optimizer.step()
        svm_loss_per_epoch  += loss.item()
        
    svm_losses.append(svm_loss_per_epoch / len(train_loader))
    print(f'Epoch {epoch+1}/{num_epochs}, SVM Loss: {svm_losses[-1]}')

 20%|██        | 1/5 [00:07<00:29,  7.36s/it]

Epoch 1/5, SVM Loss: 0.7875140408780783


 40%|████      | 2/5 [00:14<00:21,  7.28s/it]

Epoch 2/5, SVM Loss: 0.5038530819340429


 60%|██████    | 3/5 [00:22<00:14,  7.47s/it]

Epoch 3/5, SVM Loss: 0.4608617932525779


 80%|████████  | 4/5 [00:29<00:07,  7.51s/it]

Epoch 4/5, SVM Loss: 0.4379785778854829


100%|██████████| 5/5 [00:37<00:00,  7.52s/it]

Epoch 5/5, SVM Loss: 0.4235363910034267


In [88]:
# TODO Implement the SVM evaluation code
# Hint : use the `evaluate` function above on the test set
evaluate(svm_model, test_loader)

91.48

# NN

In [89]:
## DO NOT DELETE
class NN(nn.Module):

    def __init__(self, input_size, num_classes, hidden_size = 128):
        # TODO implement NN model here
        super().__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.output = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # TODO implement forward pass here
        x = x.view(x.size(0),-1)
        x = torch.relu(self.hidden(x))
        x = self.output(x)
        return x

In [90]:
## DO NOT DELETE
nn_model = NN(input_size = 28 * 28, num_classes= 10)
loss_fn = nn.CrossEntropyLoss()
nn_optimizer = torch.optim.Adam(nn_model.parameters(), lr=learning_rate)

In [91]:
nn_losses = []
for epoch in tqdm(range(num_epochs)):
    nn_model.train()
    nn_loss_per_epoch = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # Train NN model
        # TODO Implement the nn model training loop
        pred = nn_model(data)

        loss= loss_fn(pred, target)

        nn_optimizer.zero_grad()
        loss.backward()
        nn_optimizer.step()

        nn_loss_per_epoch += loss.item()
    nn_losses.append(nn_loss_per_epoch / len(train_loader))
    print(f'Epoch {epoch+1}/{num_epochs}, NN Loss: {nn_losses[-1]}')

 20%|██        | 1/5 [00:08<00:33,  8.35s/it]

Epoch 1/5, NN Loss: 0.26038076001793337


 40%|████      | 2/5 [00:17<00:25,  8.65s/it]

Epoch 2/5, NN Loss: 0.11336344412502164


 60%|██████    | 3/5 [00:25<00:16,  8.48s/it]

Epoch 3/5, NN Loss: 0.0798286733960709


 80%|████████  | 4/5 [00:34<00:08,  8.53s/it]

Epoch 4/5, NN Loss: 0.06089003459820504


100%|██████████| 5/5 [00:42<00:00,  8.55s/it]

Epoch 5/5, NN Loss: 0.04836411845918967


In [92]:
# TODO Implement the NN evaluation code
# Hint : use the `evaluate` function above on the test set
evaluate(nn_model, test_loader)

97.46

# CNN

In [93]:
## DO NOT DELETE
class CNN(nn.Module):

    def __init__(self, num_classes=10):
        # TODO implement CNN model here
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 7 * 7, num_classes)
        

    def forward(self, x):
        # TODO implement forward pass here
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [94]:
## DO NOT DELETE
cnn_model = CNN(num_classes= 10)
loss_fn = nn.CrossEntropyLoss()
cnn_optimizer = torch.optim.Adam(cnn_model.parameters(), lr=learning_rate)

In [95]:
cnn_losses = []
for epoch in tqdm(range(num_epochs)):
    cnn_model.train()
    cnn_loss_per_epoch = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # Train CNN model
        # TODO Implement the nn model training loop
        cnn_optimizer.zero_grad()
        pred = cnn_model(data)
        loss = loss_fn(pred, target)
        loss.backward()
        cnn_optimizer.step()
        cnn_loss_per_epoch += loss.item()
    cnn_losses.append(cnn_loss_per_epoch / len(train_loader))
    print(f'Epoch {epoch+1}/{num_epochs}, CNN Loss: {cnn_losses[-1]}')

 20%|██        | 1/5 [00:10<00:41, 10.41s/it]

Epoch 1/5, CNN Loss: 0.1789715369353627


 40%|████      | 2/5 [00:20<00:30, 10.19s/it]

Epoch 2/5, CNN Loss: 0.05470071148463904


 60%|██████    | 3/5 [00:30<00:20, 10.11s/it]

Epoch 3/5, CNN Loss: 0.04111397223396183


 80%|████████  | 4/5 [00:40<00:10, 10.13s/it]

Epoch 4/5, CNN Loss: 0.033800749572305215


100%|██████████| 5/5 [00:50<00:00, 10.12s/it]

Epoch 5/5, CNN Loss: 0.027818138798062594


In [96]:
# TODO Implement the CNN evaluation code
# Hint : use the `evaluate` function above on the test set
evaluate(cnn_model, test_loader)

98.6